In [1]:
import pandas as pd

In [2]:
def import_data():
    raw_data_df = pd.read_csv("LCL-June2015v2_1.csv", header=0)
    return raw_data_df

In [8]:
result = import_data()
result.head()
result.columns

Index(['LCLid', 'stdorToU', 'DateTime', 'KWH/hh (per half hour) '], dtype='object')

In [10]:
result['date'] = pd.to_datetime(result['DateTime'])
data = result.loc[:,['KWH/hh (per half hour) ']]
data = data.set_index(result.date)
data['KWH/hh (per half hour)'] = pd.to_numeric(data['KWH/hh (per half hour) '], downcast='float', errors='coerce')
data.head()

,KWH/hh (per half hour),KWH/hh (per half hour)
date,,
2012-11-08 10:30:00,0.003,0.003
2012-11-08 11:00:00,0.032,0.032
2012-11-08 11:30:00,0.035,0.035
2012-11-08 12:00:00,0,0.000
2012-11-08 12:30:00,0,0.000
